---

Instalar librerias necesarias (en modo silencioso *-q*):

In [1]:
!wget -q https://raw.githubusercontent.com/arqlm/arqlm.github.io/main/_static/libraries.txt
!pip install -q -r /content/libraries.txt
!pip install -q --upgrade plotly[kaleido]
!rm -r /content/libraries.txt
!rm -r /content/sample_data/ ## Esta linea no es necesaria cuando no se trabaja en colab.google

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.9 MB/s eta 0:00:00


In [77]:
## Importar librerías

import pandas as pd
import numpy as np
import base64
import datetime
import io

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py

# pio.renderers.default = "colab"
pio.renderers.default = "notebook" # usar esta linea en jupyter notebook o vscode


---
## Verificación de tendencias (derivas)

A veces existen tendencias sistemáticas en el valor de la variable. Por ejemplo, la concentración de un elemento puede cambiar sistemáticamente con la profundidad. En estos casos, podemos dejar de lado la idea de un dominio estrictamente estacionario utilizando la noción de cuasi-estacionariedad de segundo orden; es decir, podemos asumir que para cada ubicación, las propiedades estadísticas (y geológicas) permanecen homogéneas dentro de un vecindario.

Este vecindario puede definirse para que la suposición funcione; es decir, si existe una tendencia fuerte en los datos, se puede usar un vecindario lo suficientemente pequeño para asumir estacionariedad dentro de él.

<div style="background-color:#d4edda; border:1px solid #c3e6cb; color:#155724; padding:15px; border-radius:6px; margin-bottom:10px;">
  **En presencia de tendencias sistemáticas en el atributo:**: 
  <ul style="margin-top:10px;">
    <ol style="margin-top:10px;">
      <li>Graficar medias y varianzas locales en diferentes direcciones.</li>
      <li>Determinar si el cambio en la media o varianza es significativo en relación con la escala del modelo:
      <ul style="margin-top:10px;">
      <li> Si la tendencia no es significativa: usar los vecindarios locales para estimación y simulación, y asegurarse de revisar el modelo al final para ver qué tan bien sigue las tendencias.</li>
      <li> Si la tendencia sigue siendo significativa dentro del vecindario, modelar la tendencia con una interpolación suave y calcular los residuales en las ubicaciones de las muestras.</li>
       </ul>
      </li>
      <li>Determinar si dentro de un vecindario local, de tamaño suficiente para encontrar muestras para interpolar, la tendencia sigue siendo significativa:</li>
      <li>Actualizar las estadísticas y revisar.</li>
    </ol>
  </ul>
</div>

Es claro que existe una relación entre la escala de la posible tendencia y el tamaño del vecindario. Además, la disponibilidad de datos de muestra es importante, ya que se necesita encontrar suficientes muestras dentro de cada pequeño vecindario para permitir la inferencia.

Como regla general, si hay abundantes muestras, el efecto de la tendencia será despreciable, ya que existirá suficiente información para informar las estadísticas locales dentro de un vecindario pequeño. Las tendencias se vuelven (muy) problemáticas cuando existen pocos datos, por ejemplo, en etapas tempranas de exploración, donde los sondajes están muy separados.

Definir dominios estacionarios es una decisión, y una muy importante, ya que determina qué datos se utilizan para inferir sobre ubicaciones no muestreadas dentro de esa región. Además, definir los vecindarios locales cuasi-estacionarios es tan importante como la definición de los dominios.

En la práctica, los dominios son dictados por el entendimiento geológico. Se realizan análisis estadísticos y espaciales para evaluar cuán apropiado es un modelo estacionario para los datos. Para este propósito se utilizan herramientas de análisis exploratorio de datos. Si los dominios son homogéneos desde el punto de vista geológico, pero muestran tendencias, esto puede manejarse definiendo el vecindario local apropiado al hacer inferencias sobre cada ubicación.

En algunos casos, esto no es suficiente y puede ser necesario modelar y remover la tendencia, con la esperanza de que los residuales obtenidos a través de este proceso sean estacionarios y capturen parte de la estructura espacial. Sin embargo, este es un equilibrio delicado y no se dará una regla general en esta etapa.
